# Análisis exploratorio de datos
En este informe, se realizará una exploración exhaustiva de los conjuntos de datos relacionados con los `hechos` y las `víctimas`. El objetivo principal es comprender la estructura de estos datos, identificar posibles patrones y recopilar información relevante que sirva como base para la creación del panel de control en Power BI.

### Importación de librerías necesarias

In [248]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
import googlemaps
from datetime import datetime
import requests

## EDA DataFrame `homicidios_hechos_victimas`

In [249]:
# Carga del archivo
df_hhv = pd.read_csv('datasets/homicidios_hechos_victimas.csv')

### 1. Visualizamos la Estructura del DataFrame
Visualizamos las primeras filas del DataFrame

In [250]:
df_hhv.head()

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,victima,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto-auto,moto,auto,fatal,conductor,moto,masculino,19.0
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto-pasajeros,auto,pasajeros,fatal,conductor,auto,masculino,70.0
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto-auto,moto,auto,fatal,conductor,moto,masculino,30.0
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto-sd,moto,NaN,fatal,conductor,moto,masculino,18.0
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto-pasajeros,moto,pasajeros,fatal,conductor,moto,masculino,29.0


In [251]:
# Vemos la información de cada columna de nuestro DF
df_hhv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_hecho          717 non-null    object 
 1   n_victimas        717 non-null    int64  
 2   fecha             717 non-null    object 
 3   franja_hora       716 non-null    float64
 4   tipo_calle        717 non-null    object 
 5   comuna            715 non-null    float64
 6   pos_x             704 non-null    float64
 7   pos_y             704 non-null    float64
 8   participantes     717 non-null    object 
 9   victima           708 non-null    object 
 10  acusado           694 non-null    object 
 11  gravedad          717 non-null    object 
 12  rol               706 non-null    object 
 13  vehiculo_victima  708 non-null    object 
 14  sexo_victima      711 non-null    object 
 15  edad_victima      664 non-null    float64
dtypes: float64(5), int64(1), object(10)
memory u

### 2. Eliminamos columnas redundantes

In [252]:
# Eliminar la columna 'vehiculo_victima' de forma permanente
df_hhv.drop('victima', axis=1, inplace=True)

### 3. Cambiamos tipo de dato, para que facilite el análisis

Convertir las columnas a ``enteros`` y manejar los valores NaN

In [253]:
df_hhv['franja_hora'] = df_hhv['franja_hora'].astype('Int64')
df_hhv['edad_victima'] = df_hhv['edad_victima'].astype('Int64')

Convertir a tipo de datos ``categóricos``

In [254]:

df_hhv['tipo_calle'] = df_hhv['tipo_calle'].astype('category')
df_hhv['comuna'] = df_hhv['comuna'].astype('category')
df_hhv['participantes'] = df_hhv['participantes'].astype('category')
df_hhv['acusado'] = df_hhv['acusado'].astype('category')
df_hhv['gravedad'] = df_hhv['gravedad'].astype('category')
df_hhv['rol'] = df_hhv['rol'].astype('category')
df_hhv['vehiculo_victima'] = df_hhv['vehiculo_victima'].astype('category')
df_hhv['sexo_victima'] = df_hhv['sexo_victima'].astype('category')

Convertimos a tipo de dato ``datetime``

In [255]:
df_hhv['fecha'] = pd.to_datetime(df_hhv['fecha'], errors='coerce')

In [256]:
df_hhv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_hecho          717 non-null    object        
 1   n_victimas        717 non-null    int64         
 2   fecha             717 non-null    datetime64[ns]
 3   franja_hora       716 non-null    Int64         
 4   tipo_calle        717 non-null    category      
 5   comuna            715 non-null    category      
 6   pos_x             704 non-null    float64       
 7   pos_y             704 non-null    float64       
 8   participantes     717 non-null    category      
 9   acusado           694 non-null    category      
 10  gravedad          717 non-null    category      
 11  rol               706 non-null    category      
 12  vehiculo_victima  708 non-null    category      
 13  sexo_victima      711 non-null    category      
 14  edad_victima      664 non-

### 4. Tratamiento de columnas con ``valores nulos``

Tratando columnas ``franja_hora`` y `comuna`

In [257]:
nombre_columna = 'franja_hora'
filas_con_nulos = df_hhv[df_hhv[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima
536,2019-0103,1,2019-12-18,<NA>,gral paz,11.0,-58.521694,-34.594716,moto-moto,moto,fatal,conductor,moto,masculino,24


Procedemos a imputar con la ``mediana`` el dato faltante en la columna ``franja_hora``, porque preferimos preservar la fila completa, ya que ese único dato faltante no afectará en gran manera

In [258]:
mediana_edad = df_hhv['franja_hora'].median()
df_hhv['franja_hora'] = df_hhv['franja_hora'].fillna(mediana_edad)

In [259]:
nombre_columna = 'comuna'
filas_con_nulos = df_hhv[df_hhv[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima
121,2016-0151,1,2016-11-18,20,calle,NaN,NaN,NaN,peaton-sd,NaN,fatal,peatón,peatón,NaN,<NA>
141,2016-0174,1,2016-12-27,0,autopista,NaN,NaN,NaN,sd-sd,NaN,fatal,NaN,NaN,NaN,<NA>


Procedemos a eliminar estas 2 filas, ya que la información en su mayoría de columnas es nula, no aportará mucho preservar estas columnas

In [260]:
filas_a_eliminar = [121, 141]
df_hhv =df_hhv.drop(filas_a_eliminar, axis=0)

In [261]:
df_hhv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id_hecho          715 non-null    object        
 1   n_victimas        715 non-null    int64         
 2   fecha             715 non-null    datetime64[ns]
 3   franja_hora       715 non-null    Int64         
 4   tipo_calle        715 non-null    category      
 5   comuna            715 non-null    category      
 6   pos_x             704 non-null    float64       
 7   pos_y             704 non-null    float64       
 8   participantes     715 non-null    category      
 9   acusado           694 non-null    category      
 10  gravedad          715 non-null    category      
 11  rol               705 non-null    category      
 12  vehiculo_victima  707 non-null    category      
 13  sexo_victima      711 non-null    category      
 14  edad_victima      664 non-null 

In [262]:
nombre_columna = 'pos_x'
filas_con_nulos = df_hhv[df_hhv[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima
39,2016-0052,1,2016-04-20,20,autopista,13.0,NaN,NaN,moto-sd,NaN,fatal,NaN,moto,NaN,<NA>
108,2016-0136,1,2016-10-25,0,autopista,4.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,NaN,<NA>
182,2017-0042,1,2017-04-10,9,gral paz,14.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,masculino,<NA>
186,2017-0050,2,2017-04-28,11,autopista,9.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,masculino,46
187,2017-0050,2,2017-04-28,11,autopista,9.0,NaN,NaN,moto-cargas,cargas,fatal,pasajero,moto,masculino,16
188,2017-0051,1,2017-05-01,3,autopista,7.0,NaN,NaN,auto-auto,auto,fatal,conductor,auto,masculino,33
266,2017-0140,1,2017-11-19,23,autopista,4.0,NaN,NaN,moto-pasajeros,pasajeros,fatal,conductor,moto,masculino,24
327,2018-0039,1,2018-04-21,22,autopista,14.0,NaN,NaN,peaton-auto,auto,fatal,peatón,peatón,masculino,37
564,2020-0026,1,2020-05-17,6,autopista,14.0,NaN,NaN,moto-objeto fijo,objeto fijo,fatal,conductor,moto,masculino,28
578,2020-0039,1,2020-09-01,19,calle,9.0,NaN,NaN,peaton-cargas,cargas,fatal,peatón,peatón,masculino,44


In [263]:
# Nombre de la columna con nulos
nombre_columna = 'pos_x'
# Filas con nulos en la columna 'pos_x'
filas_con_nulos = df_hhv[df_hhv[nombre_columna].isnull()]
# Obtener los códigos de las filas con nulos en la columna 'pos_x'
codigos_con_nulos = filas_con_nulos['id_hecho'].tolist()
print(codigos_con_nulos)

['2016-0052', '2016-0136', '2017-0042', '2017-0050', '2017-0050', '2017-0051', '2017-0140', '2018-0039', '2020-0026', '2020-0039', '2021-0023']


In [264]:
df_original = pd.read_excel('datasets/raw/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])

In [265]:
# Filtrar la base de datos original usando los códigos con nulos
df_original_filtrado = df_original[df_original['ID'].isin(codigos_con_nulos)]

# Mostrar el DataFrame filtrado
df_original_filtrado

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20.0,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,NaN
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0.0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9.0,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",NaN,NaN,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11.0,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,NaN,NaN,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3.0,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,NaN,NaN,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23.0,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22.0,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6.0,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",NaN,NaN,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19.0,MURGUIONDO 2700,CALLE,MURGUIONDO,NaN,NaN,MURGUIONDO,9,Point (. .),.,.,PEATON-CARGAS,PEATON,CARGAS
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9.0,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,NaN,NaN,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


In [266]:
df_original.rename(columns={'ID': 'id_hecho'}, inplace=True)

In [271]:
import pandas as pd
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key='Holaaaa :)')

def actualizar_coordenadas_con_api(df_hhv, df_original):
    for index, row in df_hhv.iterrows():
        if pd.isna(row['pos_x']) or pd.isna(row['pos_y']):
            direccion_original = df_original.loc[df_original['id_hecho'] == row['id_hecho'], 'LUGAR_DEL_HECHO'].values[0]
            geocode_result = gmaps.geocode(direccion_original)
            if geocode_result:
                lat = geocode_result[0]['geometry']['location']['lng']
                lng = geocode_result[0]['geometry']['location']['lat']
                df_hhv.at[index, 'pos_x'] = lng
                df_hhv.at[index, 'pos_y'] = lat
            else:
                print(f"No se pudo obtener la ubicación para el id_hecho {row['id_hecho']}")

In [272]:
actualizar_coordenadas_con_api(df_hhv, df_original)

No se pudo obtener la ubicación para el id_hecho 2016-0052
No se pudo obtener la ubicación para el id_hecho 2017-0042
No se pudo obtener la ubicación para el id_hecho 2018-0039
No se pudo obtener la ubicación para el id_hecho 2020-0026


In [273]:
nombre_columna = 'pos_x'
filas_con_nulos = df_hhv[df_hhv[nombre_columna].isnull()]
filas_con_nulos

,id_hecho,n_victimas,fecha,franja_hora,tipo_calle,comuna,pos_x,pos_y,participantes,acusado,gravedad,rol,vehiculo_victima,sexo_victima,edad_victima
39,2016-0052,1,2016-04-20,20,autopista,13.0,NaN,NaN,moto-sd,NaN,fatal,NaN,moto,NaN,<NA>
182,2017-0042,1,2017-04-10,9,gral paz,14.0,NaN,NaN,moto-cargas,cargas,fatal,conductor,moto,masculino,<NA>
327,2018-0039,1,2018-04-21,22,autopista,14.0,NaN,NaN,peaton-auto,auto,fatal,peatón,peatón,masculino,37
564,2020-0026,1,2020-05-17,6,autopista,14.0,NaN,NaN,moto-objeto fijo,objeto fijo,fatal,conductor,moto,masculino,28
